### Day 22: Monkey Market

Link: https://adventofcode.com/2024/day/22#part2

To solve part two, we need to modify the solution from part one as follows:

1. For each buyer, store a list of prices corresponding to the last digit of the current secret and their corresponding delta to the previous price.
2. Use a sliding window to contain the last four deltas. If that sequence hasn't been seen yet, store its corresponding price in a dictionary.
3. For each buyer's sequence, add the price to a dictionary containing all sequences from all buyers.
4. Return the highest price sum from the previous dictionary.

In [7]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
from collections import deque


numbers: list[int] = []


for line in lines:
    numbers.append(int(line.strip()))


def mix(number: int, secret: int) -> int:
    return secret ^ number


def prune(secret: int) -> int:
    return secret % 16_777_216


def get_last_digit(number: int) -> int:
    return number % 10


buyers_deltas: list[list[tuple[int, int]]] = []
sequences_prices: dict[tuple[int, int, int, int], int] = {}


for number in numbers:
    secret = number
    buyer_deltas = [(get_last_digit(secret), float("inf"))]
    sequence: deque[int] = deque()
    buyer_sequences_prices: dict[tuple[int, int, int, int], int] = {}

    for _ in range(2_000):
        secret = prune(mix(secret * 64, secret))
        secret = prune(mix(secret // 32, secret))
        secret = prune(mix(secret * 2048, secret))

        price = get_last_digit(secret)
        delta = price - buyer_deltas[-1][0]
        buyer_deltas.append((price, delta))

        sequence.append(delta)

        if len(sequence) > 4:
            sequence.popleft()

        if len(sequence) == 4:
            key = tuple(sequence)

            if key in buyer_sequences_prices:
                continue

            buyer_sequences_prices[key] = price

    for key in buyer_sequences_prices:
        sequences_prices.setdefault(key, 0)
        sequences_prices[key] += buyer_sequences_prices[key]

    buyers_deltas.append(buyer_deltas)


print(max(sequences_prices.values()))